In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs
import pandas as pd
from experiments.ray.analysis import load_tasks, load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment, load_scheduling_queue
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve, SyntheticPolynomialCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from omegaconf import OmegaConf
from pathlib import Path
from collections import defaultdict

In [ ]:
from privacypacking.budget import ALPHAS

In [ ]:
block = Budget.from_epsilon_delta(epsilon=10, delta=1e-8)

In [ ]:
block.epsilon(ALPHAS[4])

In [ ]:
s = SyntheticPolynomialCurve(
                                best_alpha=6,
                                epsilon_min=0.1
                                * block.epsilon(6),
                                epsilon_left=1
                                * block.epsilon(ALPHAS[4]),
                                epsilon_right=1
                                * block.epsilon(ALPHAS[-1]),
                            )

In [ ]:
s

In [ ]:
from privacypacking.utils.zoo import zoo_df, build_zoo, build_synthetic_zoo

In [ ]:
names_and_curves = build_synthetic_zoo()
alphas_df, tasks_df = zoo_df(names_and_curves)

In [ ]:
names_and_curves[0]

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
x = np.array([ALPHAS[4], ALPHAS[6], ALPHAS[-1]])
f = interp1d(x=x, y=[0.5, 0.1, 1.0], kind='slinear')
px.line(
    x=x,
    y=f(x)
 )

In [ ]:
alphas_df.head()

In [ ]:
indx = alphas_df.groupby('task_id')['normalized_epsilons'].idxmin()
best_alpha = alphas_df.loc[indx][["task_id","alphas"]]
best_alpha = best_alpha.rename(columns={"alphas":"best_alpha"})
best_alpha

In [ ]:
alphas_df = alphas_df.merge(best_alpha, how="outer", on="task_id")

In [ ]:
alphas_df.head(2)

In [ ]:
px.line(
    # alphas_df.sort_values("alphas").query("best_alpha_x == 6 and epsilon_min == 0.01"),
    # alphas_df.sort_values("alphas").query("best_alpha_x == 6"),
    alphas_df.sort_values("alphas"),
    x="alphas",
    y="normalized_epsilons",
    color="task_name",
    log_y=True,
    log_x=True,
)

In [ ]:
indx = alphas_df.groupby('task_id')['normalized_epsilons'].idxmin()
best_alpha = alphas_df.loc[indx]
px.scatter(
    best_alpha,
    x="alphas",
    y="normalized_epsilons",
    color="task_id",
    log_y=True,
    log_x=True,
    title="Epsilon for the best alpha of each task",
)

Let's see what curves are allocated by different schedulers

In [ ]:
# 
df = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_09-58-57"))
# df = load_latest_ray_experiment("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_10-07-26")
def get_alpha_std(path):
    _, d = path.split("-")
    p = float(d.replace(".yaml", ""))
    return p
df["alpha_std"] = df["task_frequencies_path"].apply(get_alpha_std)

In [ ]:
df

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    # facet_col="data_lifetime",
    facet_row="total_tasks",
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    # facet_col="data_lifetime",
    facet_row="total_tasks",
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
# df = load_latest_scheduling_results(expname="ray/run_and_report_2022-04-05_22-11-57")
# TODO: custom load from Zoo

In [ ]:
df.columns